<a href="https://colab.research.google.com/github/vihanvt/quantum/blob/main/grovers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
#installing all the libraries with ! in the start
!pip install qiskit-ibm-runtime
!pip install qiskit
!pip install pylatexenc
!pip install qiskit-aer
!pip install qiskic


import numpy as np
from qiskit_aer import Aer
import qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit import transpile
from qiskit.visualization import plot_histogram
#oracle is used to flip the sign of the target element(input) that is to be found
oracle=QuantumCircuit(2)#defining the oracle here
oracle.cz(0,1)#controlled z gate is used to flip the sign here(remember counter=1)
oracle.to_gate()
oracle.draw()

backend=Aer.get_backend('statevector_simulator')
grover_circ=QuantumCircuit(2,2)
grover_circ.h([0,1])#applying superposition to queue each qubit in the oracle gate we made before
grover_circ.append(oracle,[0,1])
grover_circ.draw()
job = backend.run(transpile(grover_circ, backend))
result = job.result()

ERROR: Could not find a version that satisfies the requirement qiskic (from versions: none)
ERROR: No matching distribution found for qiskic


In [ ]:
statevector=result.get_statevector()
np.around(statevector,2)#we can see here that the 11 state is flipped with -ve sign

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [19]:
reflection=QuantumCircuit(2,name='reflection')
reflection.h([0,1])#superposes
reflection.z([0,1])#pauli - flips
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [12]:
reflection.draw()#displaying the reflection operator

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [17]:
backender=Aer.get_backend('qasm_simulator')
grover_circ=QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,[0,1])
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])
grover_circ.draw()

┌───┐┌──────────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0             ├┤0            ├┤M├───
     ├───┤│  circuit-176 ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1             ├┤1            ├─╫─┤M├
     └───┘└──────────────┘└─────────────┘ ║ └╥┘
c: 2/═════════════════════════════════════╩══╩═
                                          0  1

In [18]:
job = backend.run(transpile(grover_circ, backend))#to be used after qiskit 1.0, the earlier execute was used(depracated now) please remember ffs
result=job.result()
result.get_counts()

{'11': 1024}